# Clasificación por Random Forest


hacer el top 5 variables mas influyentes y en base a eso basicamente para ver si esto resplada lo que esta en el KNN

## Importar las librerías

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from imblearn.over_sampling import RandomOverSampler


## Importar el conjunto de datos

In [2]:
data = pd.read_csv('defunciones_clean.csv')

X = data.drop(['Causa_Defunción'], axis=1)
y = data['Causa_Defunción']
oversample = RandomOverSampler(sampling_strategy='minority')
X_over, y_over = oversample.fit_resample(X, y)

data = pd.concat([X_over, y_over], axis=1)


# drop columns no relevantes
data.drop('Departamento_Residencia_Madre', axis=1, inplace=True)
data.drop('Año_Registro', axis=1, inplace=True)
data.drop('Mes_Registro', axis=1, inplace=True)
data.drop('Día_Ocurrencia', axis=1, inplace=True)        # No se considera el día de ocurrencia

data.drop('Departamento_Registro', axis=1, inplace=True)

data.drop('Municipio_Registro', axis=1, inplace=True)
data.drop('Municipio_Ocurrencia', axis=1, inplace=True)          
data.drop('Municipio_Residencia_Madre', axis=1, inplace=True)

data.drop('Año_Ocurrencia', axis=1, inplace=True)
data.drop('Mes_Ocurrencia', axis=1, inplace=True)
data.drop('Departamento_Ocurrencia', axis=1, inplace=True)

data.head()

,Área_Geográfica,Asistencia_Recibida,Año_Ocurrencia,Clase_Parto,Departamento_Ocurrencia,Edad_Madre,Estado_Civil_Madre,Escolaridad_Madre,Mes_Ocurrencia,Ocupación_Madre,...,Sexo,Sitio_Ocurrencia,Tipo_Atencion,Total_Hijos_Nacidos,Total_Hijos_Nacidos_Muertos,Total_Hijos_Vivos,Via_Parto,Nacionalidad_Madre,Grupo_Etnico_Madre,Causa_Defunción
0,99,6,2009,1,7,18.0,2,99,5,99,...,1,2,1,1.269311,2.0,1.0,99,1,4,0
1,99,1,2009,1,13,36.0,2,99,6,1,...,1,9,1,1.269311,3.0,2.0,99,1,4,0
2,99,2,2009,1,2,22.0,2,99,1,99,...,1,2,1,1.269311,3.0,2.0,99,1,4,0
3,99,2,2009,1,5,26.0,2,99,5,99,...,1,2,1,1.269311,0.0,0.0,99,1,4,0
4,99,2,2009,1,7,43.0,2,99,5,99,...,2,2,1,1.269311,10.0,6.0,99,1,4,0


In [3]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)


In [5]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [6]:
classifier = RandomForestClassifier(n_estimators=10, criterion='entropy', random_state=0)
classifier.fit(X_train, y_train)


RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

In [7]:
y_pred = classifier.predict(X_test)


In [8]:
confusion_mat = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Confusion Matrix:\n", confusion_mat)
print("Accuracy:", accuracy)


Confusion Matrix:
 [[8591   27  171]
 [   0 8878    0]
 [ 487   21  327]]
Accuracy: 0.9618419630310237


In [9]:
importance_scores = classifier.feature_importances_
sorted_indices = importance_scores.argsort()[::-1]

top_five_indices = sorted_indices[:5]
top_five_variables = data.columns[top_five_indices]

print("Top five indices:\n", sorted_indices)
print("Top five variables:\n", top_five_variables)
 

Top five indices:
 [ 2  5 10  4  8  7 16 15 19 12 14 11  6  1  0 17  3  9 13 18]
Top five variables:
 Index(['Año_Ocurrencia', 'Edad_Madre', 'Semanas_Gestacion',
       'Departamento_Ocurrencia', 'Mes_Ocurrencia'],
      dtype='object')
